# Inflation vs. Wages (Overall)
2025-09-14

## 1. Import Functions and Set Variables

In [1]:
from _notebook_setup import *

hit_api = False
save_output = False

current_year= int(datetime.now().strftime("%Y"))
years = list(range(1984, current_year + 1)) # starting in 1983 instead of 1979 because CPI data is normalized to 8/1983 due to updated expenditure weights

✅ Notebook setup complete!
✅ Available APIs: bls
✅ Available libraries: pd, np, plt, sns, datetime
✅ Helper functions: save_data(), save_figure(), save_plotly_figure(), load_data()
📁 Data directory: /Users/annebode/Documents/selfevidence.github.io/data
📁 Output directory: /Users/annebode/Documents/selfevidence.github.io/data/output
📊 Ready for analysis!


## 2. Import Data

### 2.a CPI (Unadjusted)

In [2]:
# got series_ids from https://www.bls.gov/help/hlpforma.htm#OCWC
base_series_ids = {
    "CUUR": "CPI (Unadjusted)"
 }

regions = {
    "0000": "National",
}

items = {
    "SA0": "All items",
    "SA0L1E": "Core CPI (excludes food & energy)",
    "SAF": "Food and beverages",
    "SAH": "Housing",
    "SAM": "Medical care",
}

series_ids = [base_series_id + region + item for base_series_id in base_series_ids for region in regions for item in items]

if hit_api:
    df_cpi = bls.get_data(
        series_ids = series_ids,
        years = years,
    )
    df_cpi = bls.clean_data_cpi_unadjusted(df_cpi, base_series_ids, regions, items)
    if save_output:
        save_data(df=df_cpi, filename='01_inflation_data.csv')
else:
    df_cpi = load_data(filename='01_inflation_data.csv')

df_cpi.head()

📂 Loaded: /Users/annebode/Documents/selfevidence.github.io/data/output/processed_data/01_inflation_data.csv (2500 rows)


,series_id,year,period,value,data_type,region,item,date
0,CUUR0000SA0,2003,M12,184.3,CPI (Unadjusted),National,All items,2003-12-01
1,CUUR0000SA0,2003,M11,184.5,CPI (Unadjusted),National,All items,2003-11-01
2,CUUR0000SA0,2003,M10,185.0,CPI (Unadjusted),National,All items,2003-10-01
3,CUUR0000SA0,2003,M09,185.2,CPI (Unadjusted),National,All items,2003-09-01
4,CUUR0000SA0,2003,M08,184.6,CPI (Unadjusted),National,All items,2003-08-01


In [3]:
df_cpi.describe()

,year,value
count,2500.000000,2500.000000
mean,2004.336000,223.147084
std,12.031667,97.202951
min,1984.000000,101.400000
25%,1994.000000,152.900000
50%,2004.000000,208.326000
75%,2015.000000,257.402250
max,2025.000000,583.875000


### 2.b CPS Weekly Nominal Earnings

In [4]:
# series_ids obtained from: https://data.bls.gov/PDQWeb/le
series_id_dict = {
    
    # First Decile
    'LEU0252911200': {
        'description': '(unadj)- Usual weekly earnings (first decile), Employed full time, Wage and salary workers',
        'percentile': 10,
        'race': 'All'
    },
    # First Quartile
    'LEU0252911300': {
        'description': '(unadj)- Usual weekly earnings (first quartile), Employed full time, Wage and salary workers',
        'percentile': 25,
        'race': 'All'
    },
    # Second Quartile
    'LEU0252881500': {
        'description': '(unadj)- Usual weekly earnings (second quartile), Employed full time, Wage and salary workers',
        'percentile': 50,
        'race': 'All'
    },
    # Third Quartile
    'LEU0252911400': {
        'description': '(unadj)- Usual weekly earnings (third quartile), Employed full time, Wage and salary workers',
        'percentile': 75,
        'race': 'All'
    },
    # Ninth Decile
    'LEU0252911500': {
        'description': '(unadj)- Usual weekly earnings (ninth decile), Employed full time, Wage and salary workers',
        'percentile': 90,
        'race': 'All'
    },
}

if hit_api:
    df_wages = bls.get_data(
        series_ids = list(series_id_dict.keys()),
        years = years,
    )
    df_wages = bls.clean_data_weekly_nominal_earnings(df_wages, series_id_dict)
    if save_output:
        save_data(df=df_wages, filename='01_wage_data.csv')

else:
    df_wages = load_data(filename='01_wage_data.csv')

df_wages.head()

📂 Loaded: /Users/annebode/Documents/selfevidence.github.io/data/output/processed_data/01_wage_data.csv (574 rows)


,series_id,year,period,value,data_type,description,percentile,race,date
0,LEU0252911200,2003,Q04,303.0,CPS Weekly Nominal Earnings,"(unadj)- Usual weekly earnings (first decile),...",10,All,2003-10-01
1,LEU0252911200,2003,Q03,300.0,CPS Weekly Nominal Earnings,"(unadj)- Usual weekly earnings (first decile),...",10,All,2003-07-01
2,LEU0252911200,2003,Q02,301.0,CPS Weekly Nominal Earnings,"(unadj)- Usual weekly earnings (first decile),...",10,All,2003-04-01
3,LEU0252911200,2003,Q01,300.0,CPS Weekly Nominal Earnings,"(unadj)- Usual weekly earnings (first decile),...",10,All,2003-01-01
4,LEU0252911200,2002,Q04,298.0,CPS Weekly Nominal Earnings,"(unadj)- Usual weekly earnings (first decile),...",10,All,2002-10-01


In [5]:
df_wages.describe()

,year,value,percentile
count,574.000000,574.000000,574.000000
mean,2009.940767,922.968641,50.000000
std,9.663806,602.409176,28.145099
min,1984.000000,278.000000,10.000000
25%,2003.250000,456.000000,25.000000
50%,2011.000000,705.500000,50.000000
75%,2018.000000,1253.000000,75.000000
max,2025.000000,2905.000000,90.000000


## 3. Analyze Data

In [6]:
df_wages[['percentile', 'date']].groupby('percentile').min()

,date
percentile,
10,2000-01-01
25,2000-01-01
50,1984-01-01
75,2000-01-01
90,2000-01-01


### 3.a Median Wage vs. Inflation (1984 - Present Day)

In [7]:
# normalize wage data
df_1_wage = df_wages[df_wages['percentile'] == 50].copy(deep=True)
normalization_wage = df_wages[df_wages['date'] == '1984-01-01']['value'].item()
df_1_wage['wage_adj'] = df_1_wage['value'] / normalization_wage * 100
df_1_wage = df_1_wage.sort_values(by='date')
df_1_wage.head()


,series_id,year,period,value,data_type,description,percentile,race,date,wage_adj
111,LEU0252881500,1984,Q01,323.0,CPS Weekly Nominal Earnings,(unadj)- Usual weekly earnings (second quartil...,50,All,1984-01-01,100.000000
110,LEU0252881500,1984,Q02,323.0,CPS Weekly Nominal Earnings,(unadj)- Usual weekly earnings (second quartil...,50,All,1984-04-01,100.000000
109,LEU0252881500,1984,Q03,322.0,CPS Weekly Nominal Earnings,(unadj)- Usual weekly earnings (second quartil...,50,All,1984-07-01,99.690402
108,LEU0252881500,1984,Q04,335.0,CPS Weekly Nominal Earnings,(unadj)- Usual weekly earnings (second quartil...,50,All,1984-10-01,103.715170
107,LEU0252881500,1985,Q01,336.0,CPS Weekly Nominal Earnings,(unadj)- Usual weekly earnings (second quartil...,50,All,1985-01-01,104.024768


In [8]:
# normalize inflation data
df_normalization_inflation = df_cpi[df_cpi['date']=='1984-01-01'][['series_id', 'value']].reset_index(drop=True)
df_normalization_inflation.rename(columns={'value': 'baseline_cpi'}, inplace=True)

df_1_inflation = pd.merge(df_cpi, df_normalization_inflation, on = 'series_id', how = 'left')
df_1_inflation['inflation_adj'] = df_1_inflation['value'] / df_1_inflation['baseline_cpi'] * 100
df_1_inflation = df_1_inflation.sort_values(by='date').reset_index(drop=True)
df_1_inflation.head()

,series_id,year,period,value,data_type,region,item,date,baseline_cpi,inflation_adj
0,CUUR0000SAH,1984,M01,101.4,CPI (Unadjusted),National,Housing,1984-01-01,101.4,100.0
1,CUUR0000SAM,1984,M01,104.0,CPI (Unadjusted),National,Medical care,1984-01-01,104.0,100.0
2,CUUR0000SA0,1984,M01,101.9,CPI (Unadjusted),National,All items,1984-01-01,101.9,100.0
3,CUUR0000SAF,1984,M01,102.0,CPI (Unadjusted),National,Food and beverages,1984-01-01,102.0,100.0
4,CUUR0000SA0L1E,1984,M01,102.3,CPI (Unadjusted),National,Core CPI (excludes food & energy),1984-01-01,102.3,100.0


In [12]:
df_plot = df_1_inflation.copy(deep=True)
df_plot['date_str'] = pd.to_datetime(df_plot['date']).dt.strftime('%Y-%m-%d')
df_plot = df_plot.sort_values(['date_str', 'item']) 

fig = px.line(df_plot, x='date_str', y='inflation_adj', color='item')

fig.update_layout(

    template='plotly_white',
    width=1000,
    height=650,

    title={
        'text': '📈 <b>The Rising Cost of Living</b><br><span style="font-size:16px; color:#6b7280">Consumer Price Index by Category (Inflation-Adjusted)</span>',
        'x': 0.5,
        'font': {'size': 24}
    },

    xaxis=dict(
        title={'text': '', 'font': {'size': 16, 'color': '#2E4057'}},
    ),

    yaxis=dict(
        title={'text': 'Inflation-Adjusted CPI', 'font': {'size': 16, 'color': '#2E4057'}},
    ),

    # Legend - moved to top with custom title
    legend=dict(
        title={'text': 'Category', 'font': {'size': 14, 'color': '#2E4057'}},
        orientation='h',
        yanchor='top',
        y=1.0,  # Position below title
        xanchor='center',
        x=0.5,
        font={'size': 12}
    ),
    
)

fig.update_traces(
    line=dict(width=3),
    hovertemplate='<b>%{fullData.name}</b><br>' +
                  'Date: %{x}<br>' +
                  'CPI: %{y:.0f}<br>' +
                  '<extra></extra>'  # Removes trace box
)

# # Use a curated color palette
# # fig.for_each_trace(lambda t: t.update(line=dict(width=3)))

go.FigureWidget(fig)

FigureWidget({
    'data': [{'hovertemplate': '<b>%{fullData.name}</b><br>Date: %{x}<br>CPI: %{y:.0f}<br><extra></extra>',
              'legendgroup': 'All items',
              'line': {'color': '#636efa', 'dash': 'solid', 'width': 3},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'All items',
              'showlegend': True,
              'type': 'scattergl',
              'uid': '10b885c2-d65e-4fcb-8a72-62314e4337d3',
              'x': array(['1984-01-01', '1984-02-01', '1984-03-01', ..., '2025-06-01',
                          '2025-07-01', '2025-08-01'], shape=(500,), dtype=object),
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAAWUCrVRFBZx9ZQFXesfT2K1' ... '6+yHNARQSdfWTQc0DYeWG09t5zQA=='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': '<b>%{fullData.name}</b><br>Date: %{x}<br>CPI: %{y:.0f}<br><extra></extra>',
              'legendgroup': 'Core CPI (exclud

In [13]:
save_plotly_figure(
    fig=fig,
    filename='01_cpi_chart_1984',
    formats=['html'],
    subdir="figures",
    for_blog=True
)

📝 Blog version saved: /Users/annebode/Documents/selfevidence.github.io/docs/assets/charts/01_cpi_chart_1984.html
📊 Plotly figure saved: html: /Users/annebode/Documents/selfevidence.github.io/data/output/figures/01_cpi_chart_1984.html
✨ To embed in Jekyll post, use:
<iframe src="{{ site.baseurl }}/assets/charts/01_cpi_chart_1984.html" width="100%" height="700" frameborder="0"></iframe>


{'html': PosixPath('/Users/annebode/Documents/selfevidence.github.io/data/output/figures/01_cpi_chart_1984.html')}